<h1> Mixed state HMM with oSLRAU</h1>  

We use a simple mixed HMM first

In [80]:
from mixed_state_hmm import MixedHMM

In [81]:
# Define MHMM
mixed_states = [[0.0, 1.0, 0.5], [10.0, 1.0, 0.5], [20.0, 1.0, 0.5]]
mhmm = MixedHMM(mixed_states)


<h3> We now define and try to use OSLRAU on it</h3>

In [82]:
from oSLRAU_run import get_data
from spn.structure.leaves.parametric.Parametric import Gaussian, Bernoulli,  In_Latent
from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.io.Graphics import plot_spn
import numpy as np
from spn.algorithms.Inference import log_likelihood
from sklearn.model_selection import train_test_split
from spn.algorithms.oSLRAU import oSLRAU, oSLRAUParams
from spn.algorithms.RSPN import RSPN
from spn.algorithms.TransformStructure import Prune ,Prune_oSLRAU

from hmmlearn import hmm
from spn.algorithms.Inference import log_likelihood

In [83]:
n_states = 3
n_dim = 2

num_time_steps_mix = 5
num_seq_mix = 20
rspn_data_mix= np.array([mhmm.sample_obs(num_time_steps_mix) for _ in range(num_seq_mix)])
rspn_data_mix.shape

rspn_data_mix = rspn_data_mix.reshape((rspn_data_mix.shape[0], -1))
print(f"==>> rspn_data_mix.shape: {rspn_data_mix.shape}")

==>> rspn_data_mix.shape: (20, 10)


In [84]:
rspn_data_mix

array([[ 5.40951618e-01,  0.00000000e+00,  1.22011098e+01,
         0.00000000e+00,  2.03609411e+01,  0.00000000e+00,
         1.42801961e+00,  1.00000000e+00,  9.97561176e+00,
         0.00000000e+00],
       [-5.58022789e-01,  0.00000000e+00,  1.09583045e+01,
         1.00000000e+00,  1.97122954e+01,  0.00000000e+00,
         2.23204929e-01,  0.00000000e+00,  8.05934625e+00,
         0.00000000e+00],
       [ 1.16753863e-01,  1.00000000e+00,  9.16720846e+00,
         0.00000000e+00,  2.09858381e+01,  1.00000000e+00,
         2.84095627e-01,  0.00000000e+00,  7.97525585e+00,
         1.00000000e+00],
       [-2.48967406e-01,  0.00000000e+00,  7.67524104e+00,
         0.00000000e+00,  1.86114054e+01,  0.00000000e+00,
         7.16277773e-01,  1.00000000e+00,  1.18902756e+01,
         1.00000000e+00],
       [ 2.40609841e+00,  0.00000000e+00,  9.75683940e+00,
         1.00000000e+00,  2.12730204e+01,  1.00000000e+00,
        -1.20032512e+00,  1.00000000e+00,  1.00959040e+01,
         0.

In [85]:
num_variables = num_time_steps_mix * n_dim
num_latent_variables = n_states
num_latent_values = n_dim
unroll = 'backward'
full_update = False
update_leaves = True
len_sequence_varies = False
oSLRAU_params = oSLRAUParams(mergebatch_threshold=10, corrthresh=0.7, mvmaxscope=1, equalweight=True, currVals=True)


mini_batch_size = 2
update_after_no_min_batches = 2

In [101]:
def train_rspn_mix(train_data, test_data, rspn=None):

    if len(train_data.shape) == 3: train_data = np.squeeze(train_data)
    if len(test_data.shape) == 3: test_data = np.squeeze(test_data)

    if rspn is None:
    
        rspn = RSPN(num_variables=num_variables, num_latent_variables=num_latent_variables, num_latent_values=num_latent_values)
        first_mini_batch = train_data[0:mini_batch_size]
        n = first_mini_batch.shape[1]
        print(n)
        
        context = [Gaussian, Bernoulli]*(int(n/2)) # Bernoulli
        ds_context = Context(parametric_types=context).add_domains(first_mini_batch[:, :num_variables])
        spn, initial_template_spn, top_spn = rspn.build_initial_template(first_mini_batch, ds_context, len_sequence_varies)

    
    no_of_minibatches = int(train_data.shape[0] / mini_batch_size)

    print(f"no of minibatches: {no_of_minibatches}")

    
    for i in range(1, no_of_minibatches):
        mini_batch = train_data[i * mini_batch_size: (i+1) * mini_batch_size]
    
        update_template = False
        
        if i % update_after_no_min_batches == 0:
            print(i)
            update_template = True
    
        template_spn = rspn.learn_rspn(mini_batch, update_template, oSLRAU_params, unroll, full_update, update_leaves,
                                       len_sequence_varies)
           
            
    test_ll = np.mean(rspn.log_likelihood(test_data, unroll, len_sequence_varies))


    return test_ll, rspn, template_spn

<h3> </h3>

In [102]:
def avg_ll_mix(data, num_epochs=1, do_plot_spn=True):
    ll_list = []
    rspn=None
    train_data, val_data= train_test_split(data, test_size=0.1, random_state=42)
    for i in range(num_epochs):
        print(train_data.shape)
        
        
        epoch_ll, rspn, template_spn = train_rspn_mix(train_data, val_data, rspn)        
        print(f"epoch_ll: {epoch_ll}")
        
        ll_list.append(epoch_ll)

    average_ll = np.mean(np.array(ll_list))
    print(f" \n\n\n average ll: {average_ll}\n\n\n")

    if do_plot_spn:
        plot_spn(template_spn, 'rspn_final_template.pdf')
    
    return rspn

In [105]:
rspn_mix = avg_ll_mix(rspn_data_mix, do_plot_spn=True)

(18, 10)
10
==>> data.shape: (2, 10)
len(self.meta_types): 10
Building initial spn
Building initial template spn
Building top spn
no of minibatches: 9


UnboundLocalError: local variable 'mean' referenced before assignment

In [89]:
##generate test data from mixed HMM:

test_len_mix = 5
test_data_mix = np.array([mhmm.sample_obs(num_time_steps_mix) for _ in range(test_len_mix)])
test_data_mix

array([[[-0.39440553,  1.        ],
        [ 9.83185506,  0.        ],
        [20.89864507,  0.        ],
        [ 0.74944363,  1.        ],
        [ 8.96393841,  1.        ]],

       [[ 0.94599424,  1.        ],
        [10.3431683 ,  1.        ],
        [18.8171084 ,  1.        ],
        [-0.21722751,  0.        ],
        [11.20270412,  0.        ]],

       [[ 0.25659726,  0.        ],
        [11.00747205,  0.        ],
        [19.59840432,  1.        ],
        [-1.22519295,  1.        ],
        [10.54883014,  1.        ]],

       [[-1.16397914,  1.        ],
        [10.15472685,  0.        ],
        [19.36049533,  1.        ],
        [ 0.44021699,  0.        ],
        [ 9.48461854,  0.        ]],

       [[-0.79585869,  1.        ],
        [12.2889687 ,  0.        ],
        [19.23948818,  1.        ],
        [-1.93183881,  1.        ],
        [10.01962965,  0.        ]]])

In [90]:
# Avg test mhmm log likelihood

avg_ll_mix = mhmm.avg_ll(test_data_mix)
avg_ll_mix

-10.261148368145337

In [91]:
# RSPN  log likelihood

if len(test_data_mix.shape) == 3: test_data_mix = np.squeeze(test_data_mix)
print(test_data_mix.shape)
test_data_mix = test_data_mix.reshape((len(test_data_mix), -1))

rspn_test_ll_mix = np.mean(rspn_mix.log_likelihood(test_data_mix, unroll, len_sequence_varies))
rspn_test_ll_mix

(5, 5, 2)
Length of the sequence in mini_batch: 1
==>> data.shape: (5, 10)
==>> self.num_variables: 10
==>> self.len_sequence: 1
Evaluating rspn bottom up


/Users/dariyankhan/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Work (one drive)/Imperial/MIT/exchange_year/summer-UROP-knowledge-distillation/policy_stuff/policy-learn/policy-venv_new/lib/python3.10/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


-10.90474789106343